In [1]:
# Dependencies
from bs4 import BeautifulSoup
from lxml import html
import requests
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
from pymongo import MongoClient

In [2]:

# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser2 = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/justinmerryman/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [4]:
mars_info = {}
news_title = soup.find_all('div', class_= "content_title")[1].text
print(news_title)
news_p = soup.find('div', class_= "article_teaser_body").text
print(news_p)

Sensors Prepare to Collect Data as Perseverance Enters Mars' Atmosphere
Technology will collect critical data about the harsh entry environment during Perseverance’s entry next Thursday.


In [5]:

mars_info['title'] = news_title
mars_info['paragraph'] = news_p

In [6]:
url2 = 'https://space-facts.com/mars/'

In [7]:
tables = pd.read_html(url2)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [8]:
df = tables[0]
df = df.rename(columns= {0 : "Description", 1 : "Mars" })
df = df.set_index("Description")
df.head()

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [9]:
df.to_html('table.html')
table = df.to_html
mars_info['table'] = table 

In [10]:
# connect to mongo
mongodb_url = 'mongodb://localhost:27017'
mongo_client = MongoClient(mongodb_url)

In [11]:
# get handle to mongo db and create collection
mongo_db = mongo_client.mars_db
collection = mongo_db.mars
new = mongo_db.news


In [12]:
url3 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser2.visit(url3)

In [13]:
hemisphere_image_urls= []
titles = []
image_urls = []


In [14]:
html = browser2.html
soup3 = BeautifulSoup(html, 'html.parser')
hemispheres = soup3.find_all('div' , class_='item')

for hemisphere in hemispheres:
    title= hemisphere.find('h3').text
    search_link = hemisphere.find("a")["href"]  
    complete_link = "https://astrogeology.usgs.gov/" + search_link
    browser2.visit(complete_link)
    html = browser2.html
    soup3 = BeautifulSoup(html, 'html.parser')
    image_url = soup3.find('li')
    image_url = image_url.find("a")["href"]
    hemisphere_image_urls.append({'title': title, 'image_url':image_url})
    mars_info['hemispheres'] = hemisphere_image_urls
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [15]:
browser.quit()
browser2.quit()